In [1]:
import chess
import chess.pgn
import os

In [3]:
def read_games_from_file(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            try:
                game = chess.pgn.read_game(pgn_file)
                if game is None:
                    break
                games.append(game)
            except ValueError as e:
                print(f"Pominięto partię z powodu błędu: {e}")
    return games

def is_checkmate(game):
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board.is_checkmate()

def filter_checkmate_games(games, pgn_output_path):
    checkmate_games = []
    for cnt, game in enumerate(games):
        try:
            if is_checkmate(game):
                checkmate_games.append(game)
        except Exception as e:
            print(f"Pominięto partię numer {cnt} z powodu błędu: {e}")

    with open(pgn_output_path, "a") as output_file:
        for game in checkmate_games:
            output_file.write(str(game))
            output_file.write("\n\n")


data_path = "../data/raw/extended/"

files = os.listdir(data_path)
print(files)

# for file in files:
#     games = read_games_from_file(data_path + file)
#     print(file)
#     filter_checkmate_games(games, data_path + "mates.pgn")

['2001-1.pgn', '2024-1.pgn', '2013-2.pgn', '2013-1.pgn', '2016-1.pgn', '2003-056.pgn', '2005-2.pgn', '2003-34.pgn', '2000-1.pgn', '2002-1.pgn', '2022-2.pgn', '2004-2.pgn', '2015-2.pgn', '2015-1.pgn', '2023-1.pgn', '2012-1.pgn', '2019-1.pgn', '2003-1.pgn', '2001-2.pgn', '2007-2.pgn', '2000-2.pgn', '2009-2.pgn', '2012-2.pgn', '2011-2.pgn', '2020-1.pgn', '2022-1.pgn', '2003-2.pgn', '2019-2.pgn', '2011-1.pgn', '2008-1.pgn', '2007-1.pgn', '2006-1.pgn', '2002-2.pgn', '2018-1.pgn', '2006-2.pgn', '2005-1.pgn', '2017-1.pgn', '2004-1.pgn', '2009-1.pgn', '2014-2.pgn', '2021-1.pgn', '2021-2.pgn', '2023-2.pgn', '2010-2.pgn', '2020-2.pgn', 'mates.pgn', '2016-2.pgn', '2014-1.pgn', '2017-2.pgn', '2018-2.pgn', '2010-1.pgn', '2008-2.pgn']


In [5]:
data_path = "../data/raw/extended/"
games = read_games_from_file(data_path+"mates.pgn")


for cnt, game in enumerate(games):
    data = []
    board = chess.Board()
    cor = True
    kn = False

    for move in game.mainline_moves():
        if move in list(board.legal_moves):
            line = board.board_fen()
            turn = "True" if board.turn else "False"
            if move.promotion in [chess.KNIGHT, chess.BISHOP, chess.ROOK]:
                move.promotion = chess.QUEEN
                kn = True
            line+= ","+turn+","+move.uci()+"\n"
            data.append(line)
            board.push(move)
            if kn:
                break
        else:
            cor = False
            break
    
    if cor:
        with open(data_path+"fen_data.txt", "a") as f:
            f.writelines(data)
    if cnt % 1000 == 0:
        print(f"{cnt}/{len(games)}")

0/51366
1000/51366
2000/51366
3000/51366
4000/51366
5000/51366
6000/51366
7000/51366
8000/51366
9000/51366
10000/51366
11000/51366
12000/51366
13000/51366
14000/51366
15000/51366
16000/51366
17000/51366
18000/51366
19000/51366
20000/51366
21000/51366
22000/51366
23000/51366
24000/51366
25000/51366
26000/51366
27000/51366
28000/51366
29000/51366
30000/51366
31000/51366
32000/51366
33000/51366
34000/51366
35000/51366
36000/51366
37000/51366
38000/51366
39000/51366
40000/51366
41000/51366
42000/51366
43000/51366
44000/51366
45000/51366
46000/51366
47000/51366
48000/51366
49000/51366
50000/51366
51000/51366


In [11]:
changed = []

with open(data_path+"fen_data.txt", "r") as f:
    fens = f.readlines()
nums = [2, 3, 4, 5, 6, 7, 8]
lens = 71

for line in fens:
    fen, turn, move = line.split(",")
    for num in nums:
        fen = fen.replace(str(num), "1" * num)
    changed.append(fen+","+turn+","+move)
    assert len(fen) == lens
with open(data_path+"fen_data_ones", "w") as f:
    f.writelines(changed)

In [13]:
with open(data_path+"fen_data_ones.txt", "r") as f:
    fens = f.readlines()

for cnt, fen in enumerate(fens):
    if "k" not in fen or "K" not in fen:
        print (fen)
        print (cnt)

In [4]:
import json
import csv
prep_tokenized_path = "../data/prep/tokenized_ones_enc.csv"
to_tokenize_path = "../data/raw/extended/fen_data_ones.txt"
vocab_src_path = "../src/nlp/fen_vocab_src.json"
vocab_tar_path = "../src/nlp/vocab_enc.json"

with open(vocab_tar_path, "r") as f:
    vocab_tar = json.load(f)

with open(vocab_src_path, "r") as f:
    vocab_src = json.load(f)

with open(to_tokenize_path, "r") as f:
    lines = f.readlines()

with open(prep_tokenized_path, "w") as file:
    writer = csv.writer(file)
    writer.writerow(['sequence', 'target_from', 'target_to'])
    for idx, line in enumerate(lines):
        seq_token = [vocab_src["SOS"]]
        # tar_token = [vocab_tar["SOS"]]
        tar_token = []
        seq, turn, tar = line.split(",")
        # seq_elems = seq.split(" ")
        word = ""
        # seq_elems[1].replace("w", "wh")
        # seq_elems[1].replace("b", "bl")
        # seq_elems[2].replace("K", "Ki")
        # seq_elems[2].replace("Q", "Qu")
        # seq_elems[2].replace("k", "ki")
        # seq_elems[2].replace("q", "qu")
        for elem in seq:
            seq_token.append(vocab_src[elem])
        seq_token.append(vocab_src[turn])
        seq_token.append(vocab_src["EOS"])
        
        first, sec = tar[0:2], tar[2:4]
        tar_token.append(vocab_tar[first])
        tar_token.append(vocab_tar[sec])
        # tar_token.append(vocab_tar[th])
        # tar_token.append(vocab_tar[fo])
        # tar_token.append(vocab_tar["EOS"])
        writer.writerow([seq_token, tar_token])